# LangSmith Tutorial: Building an Advanced Flashcards Generator From Text Input Wit LangSmith

## Table of contents

1. Why LangSmith?
2. LangSmith vs. LangChain
3. Setting up LangSmith
4. LangSmith platform overview
5. Getting started with LangSmith Python SDK
   1. Setting up tracing
   2. Creating a LangSmith client and running an initial prompt
   3. Creating an unlabeled dataset in LangSmith
6. Evaluating LLMs in LangSmith
   1. Basic evaluation types for unlabeled datasets
   2. Creating labeled datasets from various sources
   3. Writing custom evaluations
7. Conclusion

## Why LangSmith?

Over 300k gpts in gpt store.
Most of them don't work as expected even though powered by gpt-4
we are not seeing gpts going viral
The reason is generic gpts are unreliable. 
The model behind them is general purpose
Fine-tuning their outputs, narrowing down their responses to specific niches are not easy
Gpts can't do this. They can't verify their own output.
Once output is out, it is displayed to the user
To improve the output, user either has to say the gpt made a mistake and hope for the best that it understands its fault and fixes it
That's why LangSmith is gaining so much popularity
It is an end-to-end platform for building, testing and deploying LLM applications. 
This tutorial covers how to build an LLM application using it

## LangSmith vs. LangChain

## Setting up LangSmith

## LangSmith platform overview

## Getting started with LangSmith Python SDK

### Setting up tracing

### Creating a LangSmith client and running an initial prompt

### Creating an unlabeled dataset in LangSmith

## Evaluating LLMs in LangSmith

### Basic evaluators for unlabeled datasets

### Creating labeled datasets from various sources

### Writing custom evaluators

## Conclusion